# 15.PPO（Proximal Policy Optimization）：近端策略优化
> **Proximal Policy Optimization (PPO) 算法** 在 2017 年由 John Schulman 等人提出：[Proximal Policy Optimization Algorithms](https://arxiv.org/abs/1707.06347)
> **TRPO** 计算的复杂性较高，尤其是在需要计算海森矩阵和执行二阶优化时，导致其不适用于计算资源有限的情况。**PPO** 基于 **TRPO**  的思想，但是其算法实现更加简单。并且大量的实验结果表明，与 **TRPO**  相比，**PPO** 能学习得一样好（甚至更快），这使得 **PPO** 成为非常流行的强化学习算法。

## 15.1 PPO 算法原理
> **TRPO** 使用 **泰勒展开近似、共轭梯度、线性搜索等** 方法直接求解：
$$\max_{\theta} \mathbb{E}_{s\sim\nu^{\pi_{\theta_k}}}\mathbb{E}_{a\sim\pi_{\theta_k}(\cdot|s)}\left[\frac{\pi_{\theta^{\prime}}(a|s)}{\pi_{\theta_k}(a|s)}A^{\pi_{\theta_k}}(s,a)\right]$$
$$\mathbb{E}_{s\sim\nu^{\pi_{\theta_k}}}[D_{KL}(\pi_{\theta_k}(\cdot|s),\pi_{\theta^{\prime}}(\cdot|s))]\leq \delta$$

- **PPO** 采用了相对简单且高效的 ***方法*** 来实现这一目标，避免了 **TRPO** 中计算海森矩阵和执行二阶优化的复杂性。
- ***方法*** 主要有两种形式：**Clipped Surrogate Objective (剪切目标函数) 、 Adaptive KL Penalty (自适应 KL 惩罚)** 

### Clipped Surrogate Objective (剪切目标函数)

### Adaptive KL Penalty (自适应 KL 惩罚)

## 15.2 PPO 代码实践
> 仍在 **车杆、倒立摆** 两个环境中测试 **PPO 算法**
> 大量实验表明，**PPO-Clip** 总是比 **PPO-Penalty** 表现得更好。因此下面专注于 **PPO-Clip** 的代码实现: